In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('fede'))
if module_path not in sys.path:
    sys.path.append(module_path)
# import pandas as pd
from client import Client
from supported_modles import Supported_modles
import utils
from fedavg import Fedavg
from sklearn.utils.class_weight import compute_sample_weight
import numpy as np
import random
from copy import deepcopy
import statistics
import matplotlib.pyplot as plt

In [2]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [3]:
selected_model = Supported_modles.NN_classifier

In [6]:
clients, test_x, test_y = utils.set_data_mock(selected_model,True, False)

Creating node1.
Creating node2.
Creating node3.
Creating node4.


In [7]:
epochs = 30
f1_before = []
for client in clients:
    client.init_empty_model(epochs=epochs)
    try:
        client.train_model(epochs=epochs)
    except ValueError:
        score = 0
        print(client.name)
        f1_before.append(score)
        continue 
    try: 
        score = client.test_model_f1(y_test=test_y, X_test=test_x)
    except IndexError:
        score = 0
    f1_before.append(score)
    print(score)

0.0
0.6880129890357812
0.0


In [8]:
fedavg = Fedavg("global", selected_model)

Waitiing for a Connection..


In [9]:
model = clients[1].model

In [10]:
###First learn model on clients:
def plot(epochs, split, rounds):
    epochs = epochs
    scores = []

    for _ in range(10):
        round_scores = []
        score = 0
        round_weights = []
        dataset_size = 0
        applicable_models = []
        first = True

        for client in clients:
            client.init_empty_model(epochs)
            X_train, X_test, y_train, y_test = client.split_data(split)

            # X_train = client.x_chunks[0]
            # y_train = client.y_chunks[0]

            client.train_model(X_train,y_train,epochs=epochs)
            dataset_size += client.x.shape[0]
            round_weights.append(dataset_size)
            applicable_models.append(client.model)
            if first:
                fedavg.init_global_model(client.model)
                first = False
            # print('.')

        round_weights = np.array(round_weights) / dataset_size # calculate weight based on actual dataset size
        fedavg.update_global_model(applicable_models, round_weights, selected_model)

        score = fedavg.test_model_f1(y_test=test_y, X_test=test_x)
        round_scores.append(score)

        number_of_rounds = rounds
        max_score = 0
        optimal_model = None


        for round in range(number_of_rounds):

            applicable_clients = random.sample((clients), len(clients))
            applicable_models = []
            applicable_name = []
            round_weights = []
            dataset_size = 0
            

            for client in applicable_clients:
                # print(f'.', end='')

                X_train, X_test, y_train, y_test = client.split_data(split)

                # X_train = client.x_chunks[round + 1]
                # y_train = client.y_chunks[round + 1]
            
                dataset_size += X_train.shape[0]
                sample_weights = compute_sample_weight('balanced', y=y_train)

                client.load_global_model(fedavg.model)
                client.train_local_agent(X_train, y_train, epochs, sample_weights)
                round_weights.append(X_train.shape[0])
                applicable_models.append(client.model)


            round_weights = np.array(round_weights) / dataset_size
            fedavg.update_global_model(applicable_models, round_weights, selected_model)

            score = fedavg.test_model_f1(y_test=test_y, X_test=test_x)
            round_scores.append(score)
            # print(score, end=' ')
            if score > max_score:
                # print(score)
                max_score = score
                optimal_model = deepcopy(fedavg.model)
        print(score)
        print('--------------------')
        scores.append(round_scores)

    #print(f'epochs: {epochs} split = {split} --- {statistics.mean(scores)} stdev {statistics.stdev(scores)}')
    plt.rcParams["figure.figsize"] = (10,10)
    for x in scores:
        plt.plot(x)

    plt.xlabel("Number of Rounds", fontsize=16)
    plt.ylabel("F1-Score", fontsize=16)
    plt.xticks(range(0,rounds+1,5))
    plt.xticks(size = 16)
    plt.yticks(size = 16)
    # plt.title('UNSW - SGD-LR, E=' +str(epochs)+', B=0.1, C=5')
    plt.show()


In [11]:
###First learn model on clients:
def find_scores(epochs, split, rounds):
    epochs = epochs
    scores = []

    for _ in range(5):
        score = 0
        round_weights = []
        dataset_size = 0
        applicable_models = []
        first = True

        for client in clients:
            client.init_empty_model(epochs)
            X_train, X_test, y_train, y_test = client.split_data(split)

            client.train_model(X_train,y_train,epochs=epochs)
            dataset_size += client.x.shape[0]
            round_weights.append(dataset_size)
            applicable_models.append(client.model)
            if first:
                fedavg.init_global_model(client.model)
                first = False
            # print('.')

        round_weights = np.array(round_weights) / dataset_size # calculate weight based on actual dataset size
        fedavg.update_global_model(applicable_models, round_weights, selected_model)

        score = fedavg.test_model_f1(y_test=test_y, X_test=test_x)

        number_of_rounds = rounds
        max_score = 0
        optimal_model = None


        for round in range(number_of_rounds):

            applicable_clients = random.sample((clients), len(clients))
            applicable_models = []
            applicable_name = []
            round_weights = []
            dataset_size = 0
            

            for client in applicable_clients:
                # print(f'.', end='')

                X_train, X_test, y_train, y_test = client.split_data(split)
            
                dataset_size += X_train.shape[0]
                sample_weights = compute_sample_weight('balanced', y=y_train)

                client.load_global_model(fedavg.model)
                client.train_local_agent(X_train, y_train, epochs, sample_weights)
                round_weights.append(X_train.shape[0])
                applicable_models.append(client.model)


            round_weights = np.array(round_weights) / dataset_size
            fedavg.update_global_model(applicable_models, round_weights, selected_model)

            score = fedavg.test_model_f1(y_test=test_y, X_test=test_x)
            print(score, end=' ')
            if score > max_score:
                # print(score)
                max_score = score
                optimal_model = deepcopy(fedavg.model)
        print(score)
        print('--------------------')
        scores.append(score)

    print(f'epochs: {epochs} split = {split} --- {statistics.mean(scores)} stdev {statistics.stdev(scores)}')
    # plt.rcParams["figure.figsize"] = (10,10)
    # for x in scores:
    #     plt.plot(x)

    # plt.xlabel("Number of Rounds", fontsize=16)
    # plt.ylabel("F1-Score", fontsize=16)
    # plt.xticks(range(0,rounds+1,5))
    # plt.xticks(size = 16)
    # plt.yticks(size = 16)
    # plt.title('UNSW - LR-SGD, E=' +str(epochs)+', B=' + str(split) +', C=5')
    # plt.show()


In [12]:
#plot(20,0.9,9)

In [13]:
#plot(20,0.8,9)

In [14]:
#plot(10,0.8,19)

In [15]:
#plot(10,0.8,19)

In [18]:
find_scores(5, 0.9, 9)
# find_scores(10, 0.8, 5)
# find_scores(20, 0.9, 10)
# find_scores(20, 0.8, 10)

0.2812977188902201 0.6317453191927128 0.5241102413041724 0.4777967371400033 0.6541621054761882 0.7985055439498634 0.7900653876997101 0.8081705962038996 0.8099179772744376 0.8099179772744376
--------------------
0.0016348328383422794 0.0 0.28261864794750496 0.7865972306325945 0.7863037803485862 0.8174499743076086 0.7898759698890258 0.8274848977603909 0.797904312618948 0.797904312618948
--------------------
0.7288514099060064 0.6686617124041514 0.4841841333651067 0.49613502734262577 0.6640795492937169 0.7957989546694718 0.8095123310785363 0.8285747581866916 0.8222925741670832 0.8222925741670832
--------------------
0.006349895525223195 0.0022667067209219756 0.6619083328063027 0.8259288992562748 0.8099295108002598 0.7843072091458196 0.6603853056440684 0.6664661380001823 0.815239218436235 0.815239218436235
--------------------
0.0 0.652244356887727 0.6586905339917002 0.7342853930057349 0.7772737525375304 0.8140125663956425 0.8039619480858782 0.8233580743413076 0.8038679488932439 0.80386794